## Adapt section 5, *Learning to balance*, using the Cart Pole environment of Gymnasium

### Main steps to take
- gather samples of the system's dynamics by running a random policy
- learn a local linear model of the system's dynamic around the point $\theta = 0$
- modify the reward function in equation (4) to reflect that in our problem the acceleration input can take only 2 values
- compute a balancing policy by leveraging the local linear model and reward function (e.g. LQR controller)

In [ ]:
import gymnasium as gym
import numpy as np
import pandas as pd

In [ ]:
env = gym.make("CartPole-v1")
# an observation is a tuple (Cart Position, Cart Velocity, Pole Angle, Pole Angular Velocity)
print(env.observation_space)
# there are two possible actions, 0 and 1, which correspond to pushing the cart left or right
print(env.action_space)

In [ ]:
def generate_sample_of_system_dynamics(env, nb_samples):
  """ 
  Returns: a dataframe containing predictors and outcomes
  """
  
  samples = []
  s, _ = env.reset()
  for _ in range(nb_samples):
    a = np.random.randint(0, 2)
    next_s, _, d, _, _ = env.step(a)
    samples.append({
      'x': s[0], 
      'x_dot': s[1], 
      'theta': s[2], 
      'theta_dot': s[3], 
      '2u - 1': 2 * a - 1, # mapping inputs to {-1, 1} rather than {0, 1} is important, because otherwise the linear regression needs a non-zero constant to fit well
      'evolution_x': next_s[0], 
      'evolution_x_dot': next_s[1], 
      'evolution_theta_dot': next_s[3]})
    if d:
      s, _ = env.reset()
    else:
      s = next_s
  return pd.DataFrame(samples)

In [ ]:
env = gym.make("CartPole-v1")
samples = generate_sample_of_system_dynamics(env, 10000)
print(samples.describe())

## Fitting a linear model for the system's dynamics around $\theta=0$

Under the assumption thata the pole's angular velocity is a linear
function of the state and action we use linear regression to find
the coefficients based on sample data

In [ ]:
from statsmodels.api import OLS

small_angle_samples = samples[np.abs(samples["theta"]) < 0.05]
predictors = small_angle_samples[['x', 'x_dot', 'theta', 'theta_dot', '2u - 1']]
outcomes = ['evolution_x_dot', 'evolution_theta_dot', 'evolution_x']
for o in outcomes:
  model = OLS(small_angle_samples[o], predictors)
  fit = model.fit()
  print(f"{o} linear model:")
  print(fit.params)
  print(f"Mean squared error of {o} fit", fit.mse_resid)

## Computing a state-feedback controller according to a discrete-time linear quadratic regulator design

In [ ]:
from control import dlqr

dT = 0.02
A = np.array([
  [1, dT, 0, 0], #x
  [0, 1, -0.014107, 0], #x_dot
  [0, 0, 1, dT], #theta
  [0.000031, 0.000019, 0.315099, 0.999998] #theta_dot
])
B = np.array([
  [0], #x
  [0.195111], #x_dot
  [0], #theta
  [-0.292548] #theta_dot
])
Q = np.diag([125, 50, 1200, 25])
R = np.diag([1.5])

K, S, E = dlqr(A, B, Q, R)
print(K)

## Trying out the learned control policy

In [ ]:
# env = gym.make("CartPole-v1", render_mode='human')
# s, _ = env.reset()
# while True:
#   print(np.dot(-K, s))
#   a = 0 if np.dot(-K, s) < 0.5 else 1
#   s, _, dead, trunc, _ = env.step(a)
#   if dead or trunc:
#     break

In [ ]:
# env.close()

## 3. Pick a robot arm description and load it in PyBullet

In [4]:
import pybullet


pybullet build time: Nov 29 2024 16:26:49


We choose the edo arm for its simplicity of use and minimal DOF (6). But as we actually want to use a simpler robot, with two moving joints, and fixed to a wall rather than to the ground. We therefore use a modified urdf file from the beginning.

In [1]:
import os

from robot_descriptions import edo_description
urdf_file_path = edo_description.URDF_PATH

def make_simpler_urdf(save_to_path):
    """
    Creates a new URDF file containing the robot arm but rotated like fixed on a wall, and with only the last two joints moving.
    
    Args:
        save_to_path (str): The path where the URDF file should be saved.
    """
    # The content of the simpler URDF file with just the <robot> tag
    urdf_content = """ <robot
  name="edo_sim_simpler">
  <link
    name="base_link">
    <inertial>
      <origin
        xyz="0.0617583602130883 0.437262464550775 -0.00395442197852672"
        rpy="0 -1.57079 0" />
      <mass
        value="0.0785942338762368" />
      <inertia
        ixx="0.0123841200738068"
        ixy="-0.000187984913202787"
        ixz="-1.32683892634308E-06"
        iyy="7.0169034503364E-05"
        iyz="-9.17416945099319E-05"
        izz="0.0123862261905614" />
    </inertial>
    <visual>
      <origin
        xyz="0 0 0"
        rpy="0 -1.57079 0" />
      <geometry>
        <mesh
          filename="package://edo_sim/meshes/base_link.STL" />
      </geometry>
      <material
        name="">
        <color
          rgba="0.792156862745098 0.819607843137255 0.933333333333333 1" />
      </material>
    </visual>
    <collision>
      <origin
        xyz="0 0 0"
        rpy="0 -1.57079 0" />
      <geometry>
        <mesh
          filename="package://edo_sim/meshes/base_link.STL" />
      </geometry>
    </collision>
  </link>
  <link
    name="link_1">
    <inertial>
      <origin
        xyz="-0.00457048841401064 0.303831811417004 -0.00202866410579916"
        rpy="0 0 0" />
      <mass
        value="0.0785942338762368" />
      <inertia
        ixx="0.0123841200738068"
        ixy="0.000187984913202727"
        ixz="-1.32683892634271E-06"
        iyy="7.01690345033622E-05"
        iyz="9.17416945099368E-05"
        izz="0.0123862261905615" />
    </inertial>
    <visual>
      <origin
        xyz="0 0 0"
        rpy="0 0 0" />
      <geometry>
        <mesh
          filename="package://edo_sim/meshes/link_1.STL" />
      </geometry>
      <material
        name="">
        <color
          rgba="0.792156862745098 0.819607843137255 0.933333333333333 1" />
      </material>
    </visual>
    <collision>
      <origin
        xyz="0 0 0"
        rpy="0 0 0" />
      <geometry>
        <mesh
          filename="package://edo_sim/meshes/link_1.STL" />
      </geometry>
    </collision>
  </link>
  <joint
    name="joint_1"
    type="fixed">
    <origin
      xyz="0.057188 0.0059831 0.13343"
      rpy="0 1.57079 -1.57079" />
    <parent
      link="base_link" />
    <child
      link="link_1" />
    <axis
      xyz="0 1 0" />
  </joint>
  <link
    name="link_2">
    <inertial>
      <origin
        xyz="-0.0168406485709407 0.071318237296396 -0.0876822373080704"
        rpy="0 0 0" />
      <mass
        value="0.0785942338762368" />
      <inertia
        ixx="0.0122070242873091"
        ixy="0.000930596667670934"
        ixz="-0.0011486325666074"
        iyy="0.007576065624266"
        iyz="0.0059380539105297"
        izz="0.00505742538729646" />
    </inertial>
    <visual>
      <origin
        xyz="0 0 0"
        rpy="0 0 0" />
      <geometry>
        <mesh
          filename="package://edo_sim/meshes/link_2.STL" />
      </geometry>
      <material
        name="">
        <color
          rgba="0.792156862745098 0.819607843137255 0.933333333333333 1" />
      </material>
    </visual>
    <collision>
      <origin
        xyz="0 0 0"
        rpy="0 0 0" />
      <geometry>
        <mesh
          filename="package://edo_sim/meshes/link_2.STL" />
      </geometry>
    </collision>
  </link>
  <joint
    name="joint_2"
    type="fixed">
    <origin
      xyz="0 0.18967 0"
      rpy="0.94237 -0.4634 -0.11653" />
    <parent
      link="link_1" />
    <child
      link="link_2" />
    <axis
      xyz="-0.88847 0.2908 0.35504" />
  </joint>
  <link
    name="link_3">
    <inertial>
      <origin
        xyz="0.00457048841401063 0.0962524890074447 0.00395442197852662"
        rpy="0 0 0" />
      <mass
        value="0.0785942338762368" />
      <inertia
        ixx="0.0123841200738068"
        ixy="0.000187984913202718"
        ixz="1.32683892634266E-06"
        iyy="7.01690345033619E-05"
        iyz="-9.17416945099381E-05"
        izz="0.0123862261905615" />
    </inertial>
    <visual>
      <origin
        xyz="0 0 0"
        rpy="0 0 0" />
      <geometry>
        <mesh
          filename="package://edo_sim/meshes/link_3.STL" />
      </geometry>
      <material
        name="">
        <color
          rgba="0.792156862745098 0.819607843137255 0.933333333333333 1" />
      </material>
    </visual>
    <collision>
      <origin
        xyz="0 0 0"
        rpy="0 0 0" />
      <geometry>
        <mesh
          filename="package://edo_sim/meshes/link_3.STL" />
      </geometry>
    </collision>
  </link>
  <joint
    name="joint_3"
    type="fixed">
    <origin
      xyz="-0.024558 0.12737 -0.16578"
      rpy="0.97336 -0.36296 2.8253" />
    <parent
      link="link_2" />
    <child
      link="link_3" />
    <axis
      xyz="1 0 0" />
  </joint>
  <link
    name="link_4">
    <inertial>
      <origin
        xyz="-0.00422951158597114 -0.00395442197852627 0.255057059248813"
        rpy="0 0 0" />
      <mass
        value="0.0785942338762368" />
      <inertia
        ixx="0.0123841200738068"
        ixy="-1.32683892634285E-06"
        ixz="0.000187984913202718"
        iyy="0.0123862261905615"
        iyz="9.17416945099506E-05"
        izz="7.01690345033621E-05" />
    </inertial>
    <visual>
      <origin
        xyz="0 0 0"
        rpy="0 0 0" />
      <geometry>
        <mesh
          filename="package://edo_sim/meshes/link_4.STL" />
      </geometry>
      <material
        name="">
        <color
          rgba="0.792156862745098 0.819607843137255 0.933333333333333 1" />
      </material>
    </visual>
    <collision>
      <origin
        xyz="0 0 0"
        rpy="0 0 0" />
      <geometry>
        <mesh
          filename="package://edo_sim/meshes/link_4.STL" />
      </geometry>
    </collision>
  </link>
  <joint
    name="joint_4"
    type="continuous">
    <origin
      xyz="0.0088 -0.1588 0"
      rpy="-1.5708 0 0" />
    <parent
      link="link_3" />
    <child
      link="link_4" />
    <axis
      xyz="0 0 -1" />
  </joint>
  <link
    name="link_5">
    <inertial>
      <origin
        xyz="0.00422951158596777 -0.00395442197852616 -0.360352489007445"
        rpy="0 0 0" />
      <mass
        value="0.0785942338762368" />
      <inertia
        ixx="0.0123841200738068"
        ixy="1.32683892634191E-06"
        ixz="0.000187984913202582"
        iyy="0.0123862261905615"
        iyz="-9.17416945099552E-05"
        izz="7.01690345033581E-05" />
    </inertial>
    <visual>
      <origin
        xyz="0 0 0"
        rpy="0 0 0" />
      <geometry>
        <mesh
          filename="package://edo_sim/meshes/link_5.STL" />
      </geometry>
      <material
        name="">
        <color
          rgba="0.792156862745098 0.819607843137255 0.933333333333333 1" />
      </material>
    </visual>
    <collision>
      <origin
        xyz="0 0 0"
        rpy="0 0 0" />
      <geometry>
        <mesh
          filename="package://edo_sim/meshes/link_5.STL" />
      </geometry>
    </collision>
  </link>
  <joint
    name="joint_5"
    type="continuous">
    <origin
      xyz="0 0 -0.1053"
      rpy="3.1416 1.1102E-14 3.1416" />
    <parent
      link="link_4" />
    <child
      link="link_5" />
    <axis
      xyz="-1 0 0" />
  </joint>
  <link
    name="link_6">
    <inertial>
      <origin
        xyz="1.10581233290358E-05 -0.00932339385603209 6.35624315718574E-06"
        rpy="0 0 0" />
      <mass
        value="0.0279702497322662" />
      <inertia
        ixx="7.63464744598253E-06"
        ixy="1.89204959067221E-10"
        ixz="5.32970316039599E-09"
        iyy="1.45744912344428E-05"
        iyz="1.08958003890421E-10"
        izz="7.62871791498103E-06" />
    </inertial>
    <visual>
      <origin
        xyz="0 0 0"
        rpy="0 0 0" />
      <geometry>
        <mesh
          filename="package://edo_sim/meshes/link_6.STL" />
      </geometry>
      <material
        name="">
        <color
          rgba="0.792156862745098 0.819607843137255 0.933333333333333 1" />
      </material>
    </visual>
    <collision>
      <origin
        xyz="0 0 0"
        rpy="0 0 0" />
      <geometry>
        <mesh
          filename="package://edo_sim/meshes/link_6.STL" />
      </geometry>
    </collision>
  </link>
  <joint
    name="joint_6"
    type="continuous">
    <origin
      xyz="-0.0039 0 0.1636"
      rpy="-1.5708 1.249E-14 1.57079" />
    <parent
      link="link_5" />
    <child
      link="link_6" />
    <axis
      xyz="0 -1 0" />
  </joint>
</robot>
"""
    
    # Write the content to the specified file
    with open(save_to_path, 'w') as file:
        file.write(urdf_content)
    
    print(f"New URDF saved to {save_to_path}")







In [3]:
#create the new file
make_simpler_urdf(save_to_path=os.path.dirname(urdf_file_path) + '/edo_sim_simpler.urdf')
simpler_urdf_path = os.path.dirname(urdf_file_path) + '/edo_sim_simpler.urdf'

New URDF saved to /Users/theo/.cache/robot_descriptions/edo_sim/robots/edo_sim_simpler.urdf


In [5]:
#load the robot and view it
pybullet.connect(pybullet.GUI_SERVER)
pybullet.setGravity(0, 0, -9.81)
robot_id = pybullet.loadURDF(simpler_urdf_path, useFixedBase=True)

Version = 4.1 Metal - 76.3
Vendor = Apple
Renderer = Apple M1
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
b3Printf: PosixSharedMemory::releaseSharedMemory removed shared memory
b3Printf: PosixSharedMemory::releaseSharedMemory detached shared memory

b3Printf: PosixSharedMemory::releaseSharedMemory removed shared memory
b3Printf: PosixSharedMemory::releaseSharedMemory detached shared memory

MotionThreadFunc thread started


In [ ]:
def reset_to_initial_state(body_id, initial_joint_positions=None):
  if initial_joint_positions is None:
    initial_joint_positions = np.zeros((pybullet.getNumJoints(body_id)))
  for i in range(pybullet.getNumJoints(body_id)):
    pybullet.resetJointState(
      bodyUniqueId=body_id,
      jointIndex=i,
      targetValue=initial_joint_positions[i],
      targetVelocity=0
    )
    # pybullet.changeDynamics(robot_id, i, linearDamping=0.0, angularDamping=0.0) # not sure if this is important
    pybullet.setJointMotorControl2(robot_id, i, pybullet.VELOCITY_CONTROL, force=0) # apparently this is really important


## 4. Apply pybullet.TORQUE_CONTROL with PD gains to make the robot go to a desired configuration.

In [ ]:
import numpy as np
import time

def pd_control(q_desired, v_desired, q_measured, v_measured, Kp, Kd):
    """
    Compute the joint torques using PD control based on desired and measured positions and velocities.

    Parameters:
    q_desired (np.array): Desired joint positions
    v_desired (np.array): Desired joint velocities
    q_measured (np.array): Measured joint positions
    v_measured (np.array): Measured joint velocities
    Kp (np.array): Proportional gains
    Kd (np.array): Derivative gains

    Returns:
    tau (np.array): Joint torques
    """
    # Compute position and velocity errors
    position_error = q_desired - q_measured
    velocity_error = v_desired - v_measured
    
    # Compute torques using PD control
    tau = Kp * position_error + Kd * velocity_error
    return tau

def control_to_desired_configuration(robot_id, q_desired, v_desired, eps=1e-6):
  # Get the number of joints in the robot
  nq = pybullet.getNumJoints(robot_id)
  joint_indices = [i for i in range(nq) if pybullet.getJointInfo(robot_id, i)[2] == pybullet.JOINT_REVOLUTE]

  # Initialize the PD control gains (these can be tuned)
  Kp = 20 * np.ones((len(joint_indices),))  # Proportional gains
  Kd = 0.05 * Kp # Derivative gains
  for step in range(1000):
      # Get the current joint states (positions and velocities)
      joint_states = pybullet.getJointStates(robot_id, joint_indices)
      q_measured = np.array([s[0] for s in joint_states])
      v_measured = np.array([s[1] for s in joint_states])

      if np.max(np.abs(q_measured - q_desired)) < eps and np.max(np.abs(v_measured - v_desired)) < eps:
        return

      # Compute the joint torques using the PD controller
      tau = np.clip(pd_control(q_desired, v_desired, q_measured, v_measured, Kp, Kd), -50, 50)

      # Apply the torques to the robot joints
      for i in joint_indices:
        pybullet.setJointMotorControl2(
          bodyIndex=robot_id, 
          jointIndex=i, 
          controlMode=pybullet.TORQUE_CONTROL, 
          force=tau[i])

      # Step the simulation forward
      pybullet.stepSimulation()

## 5. Moving the end-effector of the robot arm along a horizontal line

The line will be defined by two points $ p_1 = (0.5, 0, 0.8) $ and $ p_2 = (0, 0.5, 0.8) $

In [ ]:
def move_end_effector_along_trajectory(robot_id, traj, eff_idx):
  for target in traj:
    q_desired = pybullet.calculateInverseKinematics(
      bodyIndex=robot_id,
      endEffectorLinkIndex=eff_idx, 
      targetPosition=target
    )
    control_to_desired_configuration(robot_id, q_desired, np.zeros_like(q_desired))


## Set robot arm to initial position for pendulum balancing experiment

In [ ]:
reset_to_initial_state(robot_id)
time.sleep(1)
p1 = np.array((0.5, 0, 0.8))
p2 = np.array((0, 0.5, 0.8))
traj = p1 + np.outer(np.linspace(0, 1, 100), p2 - p1)
move_end_effector_along_trajectory(robot_id, traj, 5)

## 6. Modify the URDF file of the robot description to add a pendulum at the end effector of the arm

In [1]:
import os

from robot_descriptions import edo_description
urdf_file_path = edo_description.URDF_PATH

def addpendulum_urdf(urdf_file_path, save_to_path):
    # Read the URDF file
    with open(urdf_file_path, 'r') as file:
        urdf_data = file.readlines()

    # Remove trailing blank lines and the </robot> tag
    while urdf_data and urdf_data[-1].strip() == "":  # Remove blank lines
        urdf_data.pop()
    # Remove the last line (`</robot>`)
    if urdf_data[-1].strip() == "</robot>":
        urdf_data.pop()


    # The pendulum components to add
    pendulum_urdf = """  <!-- Pendulum Link (Thin cylinder attached to link_6) -->
  <link name="pendulum_link">
    <inertial>
      <origin xyz="0 0 -0.125" rpy="0 0 0" />
      <mass value="0.005" />
      <inertia ixx="0.0001" ixy="0" ixz="0" iyy="0.0000" iyz="0" izz="0.0001" /> <!-- ixx =izz = m * l **2 /3, rest is 0 (source wiki)  -->
    </inertial>
    <visual>
      <origin xyz="0 0 -0.125" rpy="0 0 0" />
      <geometry>
      <cylinder length="0.25" radius="0.0025"/> <!-- Thin cylinder, length 0.25m, radius 2.5mm -->
      </geometry>
      <material name="">
        <color rgba="0.792156862745098 0.819607843137255 0.933333333333333 1" />
      </material>
    </visual>
    <collision>
      <origin xyz="0 0 -0.125" rpy="0 0 0" /> 
      <geometry>
      <cylinder length="0.25" radius="0.0025"/> <!-- Thin cylinder, length 0.25m, radius 2.5mm -->
      </geometry>
    </collision>
  </link>

  <!-- Pendulum Mass at the end of the pendulum_link -->
  <link name="pendulum_mass">
    <inertial>
      <origin xyz="0 0 -0.03" rpy="0 0 0" />
      <mass value="0.001" /> <!-- Mass m at the end of 100g -->
      <inertia ixx="0.000036" ixy="0" ixz="0" iyy="0.000036" iyz="0" izz="0.000036" /> <!-- ixx = iyy=izz = 2 *m * r **2 /5 , rest is 0 (source wiki)   -->
    </inertial>
    <visual>
      <origin xyz="0 0 -0.03" rpy="0 0 0" />
      <geometry>
        <sphere radius ="0.03" /> <!-- Small sphere representing the mass, radius of 3cm -->
      </geometry>
      <material name="">
        <color rgba="0.792156862745098 0.819607843137255 0.933333333333333 1" />
      </material>
    </visual>
    <collision>
      <origin xyz="0 0 -0.03" rpy="0 0 0" />
      <geometry>
        <sphere radius="0.03" />
      </geometry>
    </collision>
  </link>

  <!-- Pendulum Joint (connects pendulum_link to link_6) -->
  <joint name="pendulum_joint" type="fixed">
    <parent link="link_6" /> <!-- Attach to link_6 -->
    <child link="pendulum_link" />
    <origin xyz="0 0 0" rpy="0 3.1416 0" /> <!-- Positioning pendulum at link_6's end -->
    <axis xyz="0 1 0" /> <!-- Swinging along Y-axis -->
    <limit lower="-0.4" upper="0.4" effort="0" velocity="1" /> <!-- Passive joint with zero effort -->
    <dynamics damping="0.1" friction="0.01" />
  </joint>

  <!-- Pendulum Mass Joint (fixes mass to the end of the pendulum) -->
  <joint name="pendulum_mass_joint" type="fixed">
    <parent link="pendulum_link" />
    <child link="pendulum_mass" />
    <origin xyz="0 0 -0.25" rpy="0 0 0" /> <!-- Attach mass at the tip of pendulum -->
  </joint>
  """
    # Append the pendulum components to the URDF data
    urdf_data.append(pendulum_urdf)

    # Add the </robot> tag back
    urdf_data.append("</robot>\n")

    # Write the modified URDF back to the file
    with open(save_to_path, 'w') as file:
        file.writelines(urdf_data)

    print("URDF file has been updated with pendulum components.")
    print(f"New URDF saved to {save_to_path}")



In [22]:
addpendulum_urdf(simpler_urdf_path, save_to_path=os.path.dirname(simpler_urdf_path) + '/edo_sim_simpler_pendulum.urdf')

URDF file has been updated with pendulum components.
New URDF saved to /Users/theo/.cache/robot_descriptions/edo_sim/robots/edo_sim_simpler_pendulum.urdf


In [4]:
pendulum_arm_path = os.path.dirname(simpler_urdf_path) + '/edo_sim_simpler_pendulum.urdf'

In [3]:
#you can now load the updated robot 
# (I don't know how to do otherwise than restarting the kernel if I had previously launched a physics server for the previous part)
import pybullet
pybullet.connect(pybullet.GUI_SERVER)
pybullet.setGravity(0, 0, -9.81)
robot_id = pybullet.loadURDF(pendulum_arm_path, useFixedBase=True)

pybullet build time: Nov 29 2024 16:26:49


Version = 4.1 Metal - 76.3
Vendor = Apple
Renderer = Apple M1
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
b3Printf: PosixSharedMemory::releaseSharedMemory removed shared memory
b3Printf: PosixSharedMemory::releaseSharedMemory detached shared memory

b3Printf: PosixSharedMemory::releaseSharedMemory removed shared memory
b3Printf: PosixSharedMemory::releaseSharedMemory detached shared memory

MotionThreadFunc thread started


In [5]:
magical_configuration = [
  -0.4549770521882466,
  -1.3507760269345115,
  -0.5222524322868297,
   0.08290083432661455,
   0.5374766564981894,
  -0.52,
  -0.4207413199640393,
   0.0
]

magical_configuration2 = [
   0,
  -1.1,
  -0.9,
   0.083,
   0.6,
   0.1,
   0.195,
   0.0
]
def reset_to_balanced_position(robot_id, mode='human_like'):
  magic = magical_configuration
  if mode == 'human_like':
    magic = magical_configuration2
  for i in range(8):
    pybullet.resetJointState(robot_id, i, magic[i], targetVelocity=0)
    pybullet.setJointMotorControl2(robot_id, i, pybullet.VELOCITY_CONTROL, force=0)
    # pybullet.setGravity(0, 0, 0)
    # hand_idx = 5
    # target_hand_pos = [0.35, 0.5, 1]
    # reset_to_initial_state(robot_id, initial_joint_positions=[0, 0, 0, 0, 0, 0, 0, 0])

    # # target_hand_orientation = [1, 0, 0, 0]
    # # q_ = pybullet.calculateInverseKinematics(robot_id, hand_idx, targetPosition=target_hand_pos, targetOrientation=target_hand_orientation)
    # # time.sleep(1)
    # # control_to_desired_configuration(robot_id, q_, np.zeros_like(q_))

    # move_end_effector_along_trajectory(robot_id, [target_hand_pos], hand_idx)
    # pybullet.resetJointState(robot_id, hand_idx, -0.52, targetVelocity=0)
    # pybullet.setGravity(0, 0, -9.81)

In [9]:
reset_to_balanced_position(robot_id)

In [6]:
import time
reset_to_balanced_position(robot_id)
time.sleep(2)
for i in range(8):
  print(pybullet.getJointState(robot_id, i))
pybullet.setTimeStep(1. / 240)
for _ in range(200):
  pybullet.stepSimulation()
  time.sleep(1.0 / 240.0)

(0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)
(-1.1, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)
(-0.9, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)
(0.083, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)
(0.6, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)
(0.1, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)
(0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)
(0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)


In [7]:
for i in range(8):
  print(pybullet.getJointState(robot_id, i))

(3.6473362580589708, 1.3861397775467719, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)
(-3.1833183894933645, 3.183496568097953, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)
(-4.146866913129585, -3.542155072023739, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)
(22.986115411532182, -4.356639123319136, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)
(-1.7501277736247443, -1.6022467276225352, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)
(66.38772919083058, 21.544190606490528, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)
(0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)
(0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)


## 7. Adapting the solution from step 2 to the more realistic simulation

In [ ]:
from tqdm import tqdm

class EnvRealistic:
  action_space = [-1, 1]
  scale_action = 0.01
  TIME_HORIZON = 1000
  def __init__(self, robot_id, pendulum_link, pendulum_joint, line_vec):
    self.robot_id = robot_id
    self.line_vector = line_vec
    self.pendulum_link = pendulum_link
    self.pendulum_joint = pendulum_joint
    self.reset()


  def get_state(self):
    (position, _, _, _, _, _, velocity, _) = pybullet.getLinkState(self.robot_id, self.pendulum_link, computeLinkVelocity=True)
    position = np.array(position)
    velocity = np.array(velocity)
    x = position[0]
    x_dot = np.dot((velocity - self.initial_pos), self.line_vector)
    theta = pybullet.getJointState(robot_id, self.pendulum_joint)[0] - self.initial_theta
    theta_dot = pybullet.getJointState(robot_id, self.pendulum_joint)[1]
    print(x, x_dot,  theta, theta_dot, self.initial_pos, self.initial_theta)
    return (x.item(), x_dot, theta, theta_dot)

  def reset(self):
    """ 
    Returns state
    """
    self.timestep = 0
    reset_to_balanced_position(self.robot_id)
    self.initial_pos = np.array(pybullet.getLinkState(self.robot_id, self.pendulum_link)[0])
    self.initial_theta = pybullet.getJointState(self.robot_id, self.pendulum_joint)[0]
    return (self.initial_pos, 0, 0, 0)
  
  def step(self, action):
    """
    Returns tuple (state, dead, truncated)
    """
    position = pybullet.getJointState(robot_id, self.pendulum_joint)[0]
    target = position + (action * self.scale_action) * self.line_vector
    move_end_effector_along_trajectory(robot_id, [target], self.pendulum_joint)
    (x, x_dot, theta, theta_dot) = self.get_state()
    self.timestep += 1
    dead = False
    truncated = False
    if np.abs(theta) > np.pi / 16:
      dead = True
    if self.timestep == self.TIME_HORIZON:
      truncated = True
    return ((x, x_dot, theta, theta_dot), dead, truncated)


def generate_sample_of_system_dynamics_realistic(env, nb_samples):
  samples = []
  s = env.reset()
  for _ in tqdm(range(nb_samples)):
    a = EnvRealistic.action_space[np.random.randint(0, 2)]
    next_s, dead, trunc = env.step(a)
    samples.append({
      'x': s[0], 
      'x_dot': s[1], 
      'theta': s[2], 
      'theta_dot': s[3], 
      'u': a, # mapping inputs to {-1, 1} rather than {0, 1} is important, because otherwise the linear regression needs a non-zero constant to fit well
      'evolution_x': next_s[0], 
      'evolution_x_dot': next_s[1], 
      'evolution_theta_dot': next_s[3]})
    if dead or trunc:
      s = env.reset()
    else:
      s = next_s
    return pd.DataFrame(samples)

In [ ]:
realistic_env = EnvRealistic(robot_id, 7, 6, np.array([1, 0, 0]))
time.sleep(1)
samples = generate_sample_of_system_dynamics_realistic(realistic_env, 1000)
# print(samples.describe())